# Shallow Neural Network (Multi-Layer Perceptron)

In [1]:
# You need keras for this notebook.
# for windows OS
!py -m pip install keras
# for unix OS
#!pip install keras

You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [107]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Input, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import keras
import pickle
import numpy as np
import pandas as pd

init_notebook_mode(connected=True)
np.random.seed(42)

## Import data sources
We import crafted features from previous preprocessing steps.

In [3]:
# import data
X = np.loadtxt('./data/feature_extraction.csv',delimiter=',')
y_ = pd.read_csv('./data/y_train.csv').values # remove headers
print('X shape:',X.shape,type(X))
print('y shape:',y_.shape,type(y_))

m = X.shape[0]

X shape: (3810, 81) <class 'numpy.ndarray'>
y shape: (3810, 3) <class 'numpy.ndarray'>


## Normalise features for better training outcomes

In [ ]:
scaler = StandardScaler()
X = scalar.fit_transform(X) #scales each feature column

## OHE of y labels
While it is necessary to do one-hot encoding of labels, the tools we use here do not require OHE. StratifiedKFold directly uses the class number without requiring one-hot encoding. Similarly by using sparse_categorical_crossentropy as our model loss, we can directly use the class number. However we still need to convert class in string format into number format.

In [4]:
y = y_[:,-1] # last column is the y label

encoder = LabelEncoder()
y_multiclass = encoder.fit_transform(y)
print('y_multiclass shape:',y_multiclass.shape)
print('Encoded classes:',['{}:{}'.format(tup[0],tup[1]) for tup in enumerate(encoder.classes_)])

y_multiclass shape: (3810,)
Encoded classes: ['0:carpet', '1:concrete', '2:fine_concrete', '3:hard_tiles', '4:hard_tiles_large_space', '5:soft_pvc', '6:soft_tiles', '7:tiled', '8:wood']


## Determine set of hyperparameters for K-fold evaluation
In our model, we shall attempt to find the best of hyperparameters. The tunable hyperparameters are:
- Number of Neurons in hidden layer 1
- Dropout value (post-activation at hidden layer 1)
- Epochs for training

In Keras we can also see the validation accuracy on every epoch.

In [5]:
hidden_neurons = [150,300,500]
dropouts = [0.1,0.25,0.5]
epochs = [250,500,750]

## Create Model
We create a hidden layer with 50 neurons.

In [6]:
def create_model(hyperparams):
    # Create shallow NN model
    input_dim = X.shape[1]
    output_dim = len(encoder.classes_) # number of classes 
    hidden_dim_1 = hyperparams['hidden_layer']
    hidden_dim_2 = output_dim*5

    model = Sequential()
    model.add(Dense(hidden_dim_1, input_dim=input_dim, activation='relu'))
    model.add(Dropout(hyperparams['dropout']))
    model.add(Dense(hidden_dim_2, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))

#     model.summary()
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

## Split Train-Test Set for later evaluation

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y_multiclass, test_size=0.2)

## Train the model and evaluate each set of hyperparameters using K-Fold Cross Validation
As defined earlier, we will create various permutations of hyperparameters. Since there are 3 possible options for each adjustable hyperparameter, we will obtain a total of 27 possible cases.

In [8]:
kfold_splits = 5

# Instantiate the cross validator
skf = StratifiedKFold(n_splits=kfold_splits, shuffle=True)

val_loss_all = {}

# Enumerate over all possible combos
for i,neurons in enumerate(hidden_neurons):
    for j,dropout in enumerate(dropouts):
        for k,epoch in enumerate(epochs):
            print('Model hidden neurons = {}, dropout rate = {}, # epochs = {}'.format(neurons,dropout,epoch))
        
            val_loss_kfold = []

            # Loop through the indices the split() method returns
            for index, (train_indices, val_indices) in enumerate(skf.split(X_train, y_train)):
                print("Training on fold " + str(index+1) + "/{}...".format(kfold_splits))

                # Generate batches from indices
                xtrain, xval = X[train_indices], X[val_indices]
                ytrain, yval = y_multiclass[train_indices], y_multiclass[val_indices]

                #define hyperparams set
                hyperparams = {}
                hyperparams['hidden_layer'] = neurons
                hyperparams['dropout'] = dropout
                hyperparams['epoch'] = epoch

                # Clear model, and create it
                model = None
                model = create_model(hyperparams)

                history = model.fit(xtrain, ytrain, epochs=hyperparams['epoch'], validation_data=(xval,yval),verbose=0)

                hist = history.history
                val_loss = hist['val_loss']
                loss = hist['loss']

                # for choosing best val acc
                val_loss_kfold.append(hist['val_loss'])
                print("Last training accuracy: " + str(hist['acc'][-1]) + ", last validation accuracy: " + str(hist['val_acc'][-1]))

            print('Average Loss: {}'.format(np.mean(val_loss_kfold)))
            val_loss_all[(neurons,dropout,epoch)] = np.mean(val_loss_kfold)
    
    

Model hidden neurons = 150, dropout rate = 0.1, # epochs = 150
Training on fold 1/5...
Last training accuracy: 0.7778234086364691, last validation accuracy: 0.6867862970949581
Training on fold 2/5...
Last training accuracy: 0.7968814115716044, last validation accuracy: 0.6972176760386328
Training on fold 3/5...
Last training accuracy: 0.784747847527351, last validation accuracy: 0.6584564860426929
Training on fold 4/5...
Last training accuracy: 0.7897540983606557, last validation accuracy: 0.7072368421052632
Training on fold 5/5...
Last training accuracy: 0.7906595658249954, last validation accuracy: 0.6902800648177082
Average Loss: 1.1562583283555086
Model hidden neurons = 150, dropout rate = 0.1, # epochs = 250
Training on fold 1/5...
Last training accuracy: 0.8427104722792608, last validation accuracy: 0.7292006527230168
Training on fold 2/5...
Last training accuracy: 0.8395568321829308, last validation accuracy: 0.7463175123725116
Training on fold 3/5...
Last training accuracy: 0.8

Last training accuracy: 0.8097580975931948, last validation accuracy: 0.7044334975369458
Training on fold 4/5...
Last training accuracy: 0.8102459016393443, last validation accuracy: 0.7203947368421053
Training on fold 5/5...
Last training accuracy: 0.7906595658005774, last validation accuracy: 0.7199341011597258
Average Loss: 1.1202140906832931
Model hidden neurons = 300, dropout rate = 0.25, # epochs = 250
Training on fold 1/5...
Last training accuracy: 0.8669404517453799, last validation accuracy: 0.7308319739960925
Training on fold 2/5...
Last training accuracy: 0.8666393106400502, last validation accuracy: 0.7070376432566323
Training on fold 3/5...
Last training accuracy: 0.8540385403854038, last validation accuracy: 0.7257799671592775
Training on fold 4/5...
Last training accuracy: 0.8676229508196721, last validation accuracy: 0.7203947368421053
Training on fold 5/5...
Last training accuracy: 0.8611224908557206, last validation accuracy: 0.7512355837633425
Average Loss: 1.0768899

Last training accuracy: 0.7893223819546631, last validation accuracy: 0.7422512235153363
Training on fold 2/5...
Last training accuracy: 0.8153467378168259, last validation accuracy: 0.6988543372497621
Training on fold 3/5...
Last training accuracy: 0.8081180811930309, last validation accuracy: 0.7241379310344828
Training on fold 4/5...
Last training accuracy: 0.7983606557377049, last validation accuracy: 0.7319078947368421
Training on fold 5/5...
Last training accuracy: 0.8070462925519087, last validation accuracy: 0.7116968702445117
Average Loss: 1.0598773589714925
Model hidden neurons = 500, dropout rate = 0.5, # epochs = 350
Training on fold 1/5...
Last training accuracy: 0.8443531827760183, last validation accuracy: 0.7210440457256155
Training on fold 2/5...
Last training accuracy: 0.8301189987689782, last validation accuracy: 0.7168576104746317
Training on fold 3/5...
Last training accuracy: 0.8388683886961059, last validation accuracy: 0.7372742200328407
Training on fold 4/5...


In [124]:
# convert dict of validation losses to 3d coordinate format + value
heatmap_3d = np.zeros([0,4])
for key,value in val_loss_all.items():
    heatmap_3d = np.vstack((heatmap_3d,[hidden_neurons.index(key[0]),dropouts.index(key[1]),epochs.index(key[2]),value]))

# saving val losses for various combinations
def write_pickle(file,data):
    with open(file, 'wb') as outfile:
        serialized = pickle.dumps(data)
        outfile.write(serialized)
        
def read_pickle(file):
    with open(file,'rb') as datafile:
        serialized = datafile.read()
        data = pickle.loads(serialized)
        return data
        
write_pickle('./history/val_loss.pickle',val_loss_all)

## Visualise the best hyperparameter selection
In the diagram, we want to visualise the losses in a 3d space, allowing us to quickly draw trends or get the lowest/highest losses. Drag & hold to visualise the 3d heatmap from different angles.

In [101]:
# Using plotly's Scatter3d

trace1 = go.Scatter3d(
    x=heatmap_3d[:,0],
    y=heatmap_3d[:,1],
    z=heatmap_3d[:,2],
    mode='markers',
    marker=dict(
        size=24,
        color=heatmap_3d[:,3],                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8,
        showscale=True
    )
)

data = [trace1]
layout = go.Layout(
    title='Validation Loss',
    width=800,
    height=800,
    scene = dict(
    xaxis=dict(
        ticktext=hidden_neurons,
        tickvals=[0,1,2],
        nticks=3,
        title='# Hidden Neurons'
    ),
    yaxis=dict(
        ticktext=dropouts,
        tickvals=[0,1,2],
        nticks=3,
        title='Dropout Rate'
    ),
    zaxis=dict(
        ticktext=epochs,
        tickvals=[0,1,2],
        nticks=3,
        title='# Epochs'
    )
))

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='3d-scatter-colorscale')

From the above plot, we compare between any of the 2 hyperparameter dimensions. All validation losses are referred to as K-fold validation losses. Let us explain some of the observations:

**1) dropout vs hidden neurons (@ epochs = 150)**

We observe that with increased learning capacity (hidden neurons = 500), increasing dropout rate actually reduces the validation loss. However, with lower learning capacity (hidden neurons = 150,300), increasing dropout actually harms the performance of the model by increasing the validation loss. This is expected, because with increasingly complex models, it is more likely to overfit, hence dropouts serve as regularisation parameters, which improves the model's ability to generalise. We also observe that for higher dropout rates (0.25,0.5), performance improves with increasing hidden neurons. If we try to apply more regularisation to models with lower learning capacity, we will observe higher bias and poorer performance.

**2) Epochs vs hidden neurons (@ dropout = 0.1)**

We observe that with more hidden neurons, performance is better with less training epochs. The lowest validation loss is at 350 epochs for a model with 150 hidden neurons, whereas the lowest validation loss is at 150 epochs for a model with 300 & 500 hidden neurons. With more neurons, we increase the complexity & expressiveness of the model, hence resulting in faster convergence. Any further training resulted in worsening performance (increased validation losses). Furthermore, due to the low dropout rate for this case, it is easy for the model to overfit.

**3) Epochs vs dropout (@ hidden neurons = 500)**

We see that for low dropout rates, we quickly converge and any further training hurts the performance. At increasing dropout rates, performance improved with more epochs, as more iterations are required for the model to converge. For a more complex model, regardless at any epoch counts, we observe that performance improves with increased dropout.

In [138]:
# Get best hyperparameter set based on val loss
val_loss_best,best_hyper = min(val_loss_all.values()),min(val_loss_all,key=val_loss_all.get)
print('The best combinaton belongs to {}, with a lowest val loss of {}.'.format(best_hyper,val_loss_best))

The best combinaton belongs to (300, 0.5, 350), with a lowest val loss of 1.0518331272969335.


## Retrain the best model
From the previous step, we have determined that the best model has the hyperparameters:
- Hidden Layer Neurons = 300
- Dropout = 0.5
- Epochs = 350

Now we will retrain this permutation using all available training data and visualise the process.

In [142]:
best_hyper[0]

300

In [143]:
#define hyperparams set
best_hyperparams = {}
best_hyperparams['hidden_layer'] = best_hyper[0]
best_hyperparams['dropout'] = best_hyper[1]
best_hyperparams['epoch'] = best_hyper[2]

model = create_model(best_hyperparams)
history = model.fit(X_train, y_train, epochs=hyperparams['epoch'],verbose=1)
    
# plot loss history
hist = history.history
val_loss = hist['val_loss']
loss = hist['loss']
plt.figure()
plt.plot(val_loss)
plt.plot(loss)
plt.legend(['val loss','loss'])

Epoch 1/350
3048/3048 [==============================] - 52s 17ms/step - loss: 1.8581 - acc: 0.3245
Epoch 2/350
3048/3048 [==============================] - 1s 210us/step - loss: 1.6638 - acc: 0.3990
Epoch 3/350
3048/3048 [==============================] - 1s 208us/step - loss: 1.5819 - acc: 0.4311
Epoch 4/350
3048/3048 [==============================] - 1s 210us/step - loss: 1.5340 - acc: 0.4331
Epoch 5/350
3048/3048 [==============================] - 1s 211us/step - loss: 1.5116 - acc: 0.4587
Epoch 6/350
3048/3048 [==============================] - 1s 212us/step - loss: 1.4757 - acc: 0.4656
Epoch 7/350
3048/3048 [==============================] - 1s 214us/step - loss: 1.4575 - acc: 0.4659
Epoch 8/350
3048/3048 [==============================] - 1s 212us/step - loss: 1.4380 - acc: 0.4701
Epoch 9/350
3048/3048 [==============================] - 1s 212us/step - loss: 1.4149 - acc: 0.4820
Epoch 10/350
3048/3048 [==============================] - 1s 212us/step - loss: 1.3863 - acc: 0.4865

3048/3048 [==============================] - 1s 204us/step - loss: 0.9182 - acc: 0.6640
Epoch 82/350
3048/3048 [==============================] - 1s 205us/step - loss: 0.9242 - acc: 0.6608
Epoch 83/350
3048/3048 [==============================] - 1s 205us/step - loss: 0.9145 - acc: 0.6650
Epoch 84/350
3048/3048 [==============================] - 1s 207us/step - loss: 0.9103 - acc: 0.6706
Epoch 85/350
3048/3048 [==============================] - 1s 204us/step - loss: 0.9070 - acc: 0.6562
Epoch 86/350
3048/3048 [==============================] - 1s 205us/step - loss: 0.8943 - acc: 0.6660
Epoch 87/350
3048/3048 [==============================] - 1s 204us/step - loss: 0.8941 - acc: 0.6686
Epoch 88/350
3048/3048 [==============================] - 1s 207us/step - loss: 0.8921 - acc: 0.6719
Epoch 89/350
3048/3048 [==============================] - 1s 205us/step - loss: 0.9140 - acc: 0.6627
Epoch 90/350
3048/3048 [==============================] - 1s 208us/step - loss: 0.8897 - acc: 0.6713
Epo

3048/3048 [==============================] - 1s 210us/step - loss: 0.6761 - acc: 0.7569
Epoch 162/350
3048/3048 [==============================] - 1s 208us/step - loss: 0.6849 - acc: 0.7536
Epoch 163/350
3048/3048 [==============================] - 1s 206us/step - loss: 0.6853 - acc: 0.7457
Epoch 164/350
3048/3048 [==============================] - 1s 206us/step - loss: 0.7065 - acc: 0.7470
Epoch 165/350
3048/3048 [==============================] - 1s 207us/step - loss: 0.6944 - acc: 0.7497
Epoch 166/350
3048/3048 [==============================] - 1s 206us/step - loss: 0.6769 - acc: 0.7474
Epoch 167/350
3048/3048 [==============================] - 1s 206us/step - loss: 0.7010 - acc: 0.7457
Epoch 168/350
3048/3048 [==============================] - 1s 211us/step - loss: 0.6663 - acc: 0.7612
Epoch 169/350
3048/3048 [==============================] - 1s 212us/step - loss: 0.6803 - acc: 0.7526
Epoch 170/350
3048/3048 [==============================] - 1s 208us/step - loss: 0.6967 - acc: 0

3048/3048 [==============================] - 1s 211us/step - loss: 0.5856 - acc: 0.7818
Epoch 242/350
3048/3048 [==============================] - 1s 212us/step - loss: 0.5939 - acc: 0.7789
Epoch 243/350
3048/3048 [==============================] - 1s 208us/step - loss: 0.5870 - acc: 0.7933
Epoch 244/350
3048/3048 [==============================] - 1s 206us/step - loss: 0.5921 - acc: 0.7838
Epoch 245/350
3048/3048 [==============================] - 1s 208us/step - loss: 0.5902 - acc: 0.7782
Epoch 246/350
3048/3048 [==============================] - 1s 207us/step - loss: 0.5676 - acc: 0.7897
Epoch 247/350
3048/3048 [==============================] - 1s 209us/step - loss: 0.5740 - acc: 0.7969
Epoch 248/350
3048/3048 [==============================] - 1s 209us/step - loss: 0.5748 - acc: 0.7972
Epoch 249/350
3048/3048 [==============================] - 1s 207us/step - loss: 0.5800 - acc: 0.7917
Epoch 250/350
3048/3048 [==============================] - 1s 208us/step - loss: 0.5669 - acc: 0

3048/3048 [==============================] - 1s 218us/step - loss: 0.5226 - acc: 0.8114
Epoch 322/350
3048/3048 [==============================] - 1s 221us/step - loss: 0.5235 - acc: 0.8071
Epoch 323/350
3048/3048 [==============================] - 1s 229us/step - loss: 0.5237 - acc: 0.8041
Epoch 324/350
3048/3048 [==============================] - 1s 215us/step - loss: 0.5167 - acc: 0.8133
Epoch 325/350
3048/3048 [==============================] - 1s 209us/step - loss: 0.5137 - acc: 0.8094
Epoch 326/350
3048/3048 [==============================] - 1s 209us/step - loss: 0.5416 - acc: 0.7979
Epoch 327/350
3048/3048 [==============================] - 1s 211us/step - loss: 0.5435 - acc: 0.8104
Epoch 328/350
3048/3048 [==============================] - 1s 209us/step - loss: 0.5072 - acc: 0.8143
Epoch 329/350
3048/3048 [==============================] - 1s 210us/step - loss: 0.5045 - acc: 0.8189
Epoch 330/350
3048/3048 [==============================] - 1s 211us/step - loss: 0.5346 - acc: 0

KeyError: 'val_loss'

In [ ]:
# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

## Analysis of Results
overfitting, underfitting, performance of dropout (reg)